In [ ]:
# @title
# ---------------------------------- Libraries ----------------------------------
!pip install -U bitsandbytes
!pip install -q transformers accelerate bitsandbytes torch pypdf gradio
!pip install --upgrade transformers


In [ ]:
# ---------------------------------- Imports ----------------------------------
from datasets import load_dataset
import gc
from google.colab import userdata
import gradio as gr
from huggingface_hub import login, notebook_login
from IPython.display import display, Markdown
import os
import random
import re
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline




In [ ]:
# ---------------------------------- HuggingFace Connection ----------------------------------
hf_token = os.environ.get('HF_TOKEN') or userdata.get('HF_TOKEN')

if hf_token:
    login(token=hf_token)
    print("HuggingFace login successful.")
else:
    print("HuggingFace token not found. Please set the HF_TOKEN environment variable or store it in Colab secrets.")

In [ ]:
# ---------------------------------- Use GPU ----------------------------------
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    # Set default device to GPU
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Running on CPU")

In [ ]:
# ---------------------------------- Functions ----------------------------------
# Helper function for markdown display
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))


def combine_news_text(example):
    # Handle potential None values gracefully
    title = example.get("title", "") or ""
    description = example.get("description", "") or ""

    # Add a separator for clarity
    return {"full_text": f"Title: {title}\nDescription: {description}"}


def format_model_output(output):
    # Extract the content within <think> tags as Reason
    reason_start = output.find("<think>") + len("<think>")
    reason_end = output.find("</think>")
    reason = output[reason_start:reason_end].strip()

    # Extract the content after </think> as Output
    output_start = reason_end + len("</think>")
    model_output_content = output[output_start:].strip()

    # Format the result
    reason = f"**Reason**:\n{reason}\n"
    output = f"**Output**:\n{model_output_content}"
    return reason, output


def analyze_news_sentiment(news_text, llm_pipeline):
    # Define the Prompt Template with specific instructions for the model
    test_question = f"""You are a concise Financial News Analyst.
    Analyze the provided news text. Output ONLY the three requested items below, each on a new line, prefixed with the specified label.

    1.  REASONING: Brief financial reasoning (1-2 points max) for the sentiment.
    2.  SENTIMENT: Choose ONE: Positive, Negative, Neutral.
    3.  TAG: A concise topic tag (1-3 words).

    Do NOT add any other text, greetings, or explanations.

    News Text:
    {news_text}"""

    # Format the prompt according to DeepSeek's expected input format with thinking tags
    test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"

    # Print the prompt for debugging purposes to verify what's being sent to the model
    print(f"Test prompt:\n{test_prompt}")

    # Run the model inference with specific generation parameters
    outputs = llm_pipeline(
        test_prompt,
        max_new_tokens = 4000,
        do_sample = True,
        temperature = 0.7,
        top_p = 0.9)

    # Extract the full generated text and parse it to separate reasoning from classification
    # The format_model_output function likely separates the thinking process from the final answer
    full_output = outputs[0]["generated_text"]
    reason, output = format_model_output(full_output)

    # Return both the reasoning process and the final sentiment classification
    return reason, output





In [ ]:
# ---------------------------------- Load Dataset From HuggingFace ----------------------------------
dataset_id = "britneymuller/cnbc_newsfeed"

print(f"Loading dataset from HuggingFace hub: {dataset_id}...")

news_dataset = load_dataset(dataset_id, split = "train")
print("Dataset loaded successfully!")


In [ ]:
# ---------------------------------- Inspect Data ----------------------------------
news_dataset
print("\n Dataset Features")
print(news_dataset.features)

# Convert to Pandas DataFrame
news_dataset.to_pandas()

# Apply the `combine_news_text()` function to extract the 'title' and 'description' into 'full_text'
news_dataset = news_dataset.map(combine_news_text)

print("\n--- Sample Data with 'full_text' ---")
print(news_dataset[0])

# Display the full_text of the first sample
print(news_dataset[0]["full_text"])


 Dataset Features
{'title': Value('string'), 'url': Value('string'), 'published_at': Value('string'), 'author': Value('string'), 'publisher': Value('string'), 'short_description': Value('string'), 'keywords': Value('string'), 'header_image': Value('string'), 'raw_description': Value('string'), 'description': Value('string'), 'scraped_at': Value('string')}

Map: 100%
 625/625 [00:00<00:00, 6349.94 examples/s]


--- Sample Data with 'full_text' ---
{'title': 'Santoli’s Wednesday market notes: Could September’s stock shakeout tee up strength for the fourth quarter?', 'url': 'https://www.cnbc.com/2021/09/29/santolis-wednesday-market-notes-could-septembers-stock-shakeout-tee-up-strength-for-the-fourth-quarter.html', 'published_at': '2021-09-29T17:09:39+0000', 'author': 'Michael Santoli', 'publisher': 'CNBC', 'short_description': "This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics.", 'keywords': 'cnbc, Premium, Articles, Investment strategy, Markets, Investing, PRO Home, CNBC Pro, Pro: Santoli on Stocks, source:tagname:CNBC US Source', 'header_image': 'https://image.cnbcfm.com/api/v1/image/106949602-1632934577499-FINTECH_ETF_9-29.jpg?v=1632934691', 'raw_description': '<div class="group"><p><em>This is the daily notebook of Mike Santoli, CNBC\'s senior markets commentator, with ideas about trends, stocks and market statistics.</em></p><ul><li>A muted, inconclusive bounce that has left the indexes fully within yesterday\'s low-to-high range all morning so far.</li></ul></div>', 'description': "This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics.A muted, inconclusive bounce that has left the indexes fully within yesterday's low-to-high range all morning so far.", 'scraped_at': '2021-10-30 14:11:23.709372', 'full_text': "Title: Santoli’s Wednesday market notes: Could September’s stock shakeout tee up strength for the fourth quarter?\nDescription: This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics.A muted, inconclusive bounce that has left the indexes fully within yesterday's low-to-high range all morning so far."}
Title: Santoli’s Wednesday market notes: Could September’s stock shakeout tee up strength for the fourth quarter?
Description: This is the daily notebook of Mike Santoli, CNBC's senior markets commentator, with ideas about trends, stocks and market statistics.A muted, inconclusive bounce that has left the indexes fully within yesterday's low-to-high range all morning so far.


In [ ]:
# ---------------------------------- HuggingFace Open Source LLM ----------------------------------
# Smaller DeepSeek model, that is suitable for Google Colab and reasoning tasks.
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# AutoModelForCausalLM.from_pretrained(...): Loads a pre-trained language model for tasks like text generation (e.g., ChatGPT-like behavior).
# model_id: The name or path of the model you want to load from Hugging Face (like "meta-llama/Llama-3-8b").
# torch_dtype="auto": Automatically chooses the best data type (like float16 or float32) based on your hardware for efficiency.
# load_in_4bit=True: Loads the model in 4-bit precision to save memory and run on limited hardware like free Colab GPUs.
# device_map="auto": Automatically puts the model on the right device (GPU if available, otherwise CPU).
# trust_remote_code=True: Use only with trusted models to avoid security risks.

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype = "auto",
                                             load_in_4bit = True,
                                             device_map = "auto",
                                             trust_remote_code = True)

model.eval()
print("Model loaded successfully in 4-bit!")

In [ ]:
# ---------------------------------- Load the Tokenizer ----------------------------------
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)

# ---------------------------------- Load the LLM Pipeline ----------------------------------
llm_pipeline = pipeline("text-generation", model = model, tokenizer = tokenizer, torch_dtype = "auto", device_map = "auto")
print("Text generation pipeline created successfully.")

# ---------------------------------- Prompt Engineering ----------------------------------
test_question = "Explain what electric cars are in simple terms. Keep the thinking short."
test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"
print(f"Test prompt:\n{test_prompt}")

In [ ]:
# ---------------------------------- Test the LLM ----------------------------------
outputs = llm_pipeline(test_prompt,
                       max_new_tokens = 4000,
                       do_sample = True,
                       temperature = 0.7,
                       top_p = 0.9)

outputs

In [ ]:
# ---------------------------------- Query the LLM for Reason and Output ----------------------------------
# Extract and print the generated text (only the response part)
full_output = outputs[0]["generated_text"]
reason, output = format_model_output(full_output)
print_markdown(reason + output)

**Reason:** Okay, so I need to explain what electric cars are in simple terms. Let me start by breaking down the term "electric cars." I know that "electric" means something related to electricity, like the ability to use electricity for power. So, electric cars must have some form of electricity to drive.

Now, what makes a car an electric car? I think it's about how the cars generate electricity. Maybe they use something like fuel cells or batteries. Fuel cells are like the inside of a battery, right? They use fuel to produce electricity. So, electric cars might have fuel cells instead of internal combustion engines.

I should also consider the technology involved. I've heard that some electric cars use superconductors, which are materials that conduct electricity without resistance. This might be part of the battery system. Maybe they have a lot of superconductors to store energy efficiently.

Another point is charging. Electric cars probably have an efficient charging system. If they use superconductors, it makes sense that they can charge quickly. But what about the battery capacity? They must have enough storage to keep the car moving for a while, even with low fuel levels.

I should also mention the benefits. Electric cars can be more efficient because they don't rely on internal combustion engines. They're also more environmentally friendly. Plus, they might offer features like automatic driving or even regenerative braking, which helps the car recover some energy when it stops.

Wait, what about the materials used? I think the cars are made from high-quality materials to reduce environmental impact. Maybe they're using recycled materials or have advanced engineering to keep costs low while being eco-friendly.

I should structure this in simple terms. Maybe start by saying electric cars are like cars that use electricity to power themselves. Then explain the technology, like fuel cells and superconductors. Mention the benefits like efficiency, environment, and features. Keep it short and easy to understand.

I think that covers it. Now, I'll put it together in a clear, concise way. **Output:** Electric cars are like cars that use electricity to power themselves. Instead of using fuel, they use something called fuel cells to generate electricity. These cars might have special materials like superconductors to store energy efficiently. They're more efficient and better for the environment. They might even have features like automatic driving or regenerative braking. All this makes electric cars a great choice for those who like being clean and efficient.


In [ ]:
# ---------------------------------- LLM Reasoning and Output ----------------------------------
# Print a separator line for clarity in the output
print("\n" + "=" * 30 + " TESTING ANALYSIS " + "=" * 30)


# Select a few random indices from the news dataset to test the analysis function
random_indices = random.sample(range(len(news_dataset)), 3)

# Iterate over each randomly selected index
for index in random_indices:
    # Retrieve the full text of the news item at the current index
    sample_news = news_dataset[index]["full_text"]

    # Analyze the sentiment of the sample news using the sentiment analysis function
    reason, output = analyze_news_sentiment(sample_news, llm_pipeline)

    # Print the analysis result header for the current index
    print(f"\n--- Analysis Result for Index {index} ---")

    # Display the reasoning and output in a formatted markdown style
    print_markdown(f"{reason}\n\n{output}")

    # Print a separator line for better readability between results
    print("-" * 60)

In [ ]:
# ---------------------------------- Gradio Functions ----------------------------------
def get_random_news():
    """Fetches and returns the full_text of a random news item."""
    if not news_dataset:  # Check if the news dataset is empty
        return "Error: No news items available."  # Return an error message if no news items are found
    random_index = random.randint(0, len(news_dataset) - 1)  # Generate a random index to select a news item
    news_text = news_dataset[random_index]['full_text']  # Retrieve the full text of the news item at the random index
    print(f"Fetched news item at index {random_index}")  # Print the index of the fetched news item
    return news_text  # Return the fetched news text

def perform_analysis(news_text):
    """Triggers analysis on the provided news text."""
    if not news_text or news_text.startswith("Error"):  # Check if the news text is empty or an error message
        return "Error: No news text to analyze."  # Return an error message if no valid news text is provided
    print(f"Analyzing news: {news_text[:50]}...")  # Print the first 50 characters of the news text being analyzed
    reason, output = analyze_news_sentiment(news_text, llm_pipeline)  # Analyze the sentiment of the news text
    return reason, output  # Return the reasoning and output from the analysis

In [ ]:
# ---------------------------------- Gradio UI ----------------------------------
with gr.Blocks(theme=gr.themes.Glass()) as demo:  # Create a Gradio Blocks interface with a glass theme
    gr.Markdown("""
    # DeepSeek Financial News Analyzer
    Fetches a random news item from the dataset.
    Click 'Analyze News' to get sentiment classification and the model's reasoning.

    """)

    with gr.Row():  # Create a row for buttons
        btn_fetch = gr.Button("🔄 Fetch Random News Item")  # Button to fetch a random news item
        btn_analyze = gr.Button("💡 Analyze News", variant="primary")  # Button to analyze the news

    news_display = gr.Textbox(  # Textbox to display the news item
        label="📰 News Text (Title & Description)",  # Label for the textbox
        lines=8,  # Number of lines in the textbox
        interactive=False,  # Make the textbox non-interactive
        placeholder="Click 'Fetch Random News Item' to display news."  # Placeholder text
    )
    # Creates a collapsible panel
    with gr.Accordion("🤖 Model Reason", open=True):  # Accordion for model reasoning
        analysis_display = gr.Markdown()  # Markdown display for the reasoning

    with gr.Accordion("🤖 Model Output", open=True):  # Accordion for model output
        analysis_output = gr.Markdown()  # Markdown display for the output

    # --- Event Handlers ---
    btn_fetch.click(  # Set up click event for the fetch button
        fn=get_random_news,  # Function to call when the button is clicked
        inputs=[],  # No inputs needed
        outputs=[news_display]  # Output to the news display textbox
    )

    btn_analyze.click(  # Set up click event for the analyze button
        fn=perform_analysis,  # Function to call when the button is clicked
        inputs=[news_display],  # Input from the news display textbox
        outputs=[analysis_display, analysis_output]  # Outputs to the reasoning and output displays
    )

    # Load initial news item when the app starts
    demo.load(  # Load function to run when the app starts
        fn=get_random_news,  # Function to call to get a random news item
        inputs=None,  # No inputs needed
        outputs=[news_display]  # Output to the news display textbox
    )

# --- Launch the Gradio App ---
print("Launching Gradio demo...")  # Print message indicating the app is launching
demo.launch(debug=True)  # Launch the Gradio app in debug mode